# Neural Networks - Architecture & Parameters

### Introduction

~ 10 mins

- How many layers? (depth)  
- How many units? (width)  
- How are they interconnected? 
- Weights and biases
- Activation function
- Loss function
- Dropout
- Learning rate
- Training batch size
- Training epochs 
- Momentum
- (...)  
u'U+2610 ToDo: Flesh out...

Finding the ideal architecture & Parameters for a task usually requires a good bit of experimentation!  

A detailed overview of different architecture types can be found here:  
http://www.asimovinstitute.org/neural-network-zoo/

![neuralnetworks.png](./neuralnetworks.png)


# Common Neural Network Types

15 mins



## Feed Forward Neural Network


- [  ] ToDo: add description / Graphic / Code exampe

## Recurrent Neural Network

15 mins

- [ ] ToDo: add description / Graphic / Code exampe

# Parameter Tuning Competition

15 mins

- [] ToDo: Very simple (and fast to train) Example with not very well tuned params.  Ask people to try to improve the parameter values, and submit/shout out Accuracy output

